<a href="https://colab.research.google.com/github/suyashmarathe512/suyash/blob/main/Speech_to_text_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install gradio duckduckgo_search

In [13]:
!pip install streaming_stt_nemo edge-tts langchain_community

In [ ]:
from huggingface_hub import InferenceClient
import gradio as gr
import random
import tempfile
import asyncio
from streaming_stt_nemo import Model
import edge_tts
from langchain_community.tools import DuckDuckGoSearchRun

API_URL = "https://api-inference.huggingface.co/models/"
client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.1")

# Initialize DuckDuckGo search tool
duckduckgo_search = DuckDuckGoSearchRun()

# Initialize ASR model
default_lang = "en"
engines = { default_lang: Model(default_lang) }

def transcribe(audio):
    """Transcribes the audio file to text."""
    lang = "en"
    model = engines[lang]
    text = model.stt_file(audio)[0]
    return text

def format_prompt(message, history):
    """Formats the prompt for the language model."""
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST] {message} [/INST]"
    return prompt

def generate(prompt, history, temperature=0.9, max_new_tokens=512, top_p=0.95, repetition_penalty=1.0):
    """Generates a response from the language model."""
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=random.randint(0, 10**7),
    )

    formatted_prompt = format_prompt(prompt, history)

    stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    output = ""

    for response in stream:
        output += response.token.text
        yield output

    search_result = duckduckgo_search.run(prompt)
    if search_result:
        yield search_result
    else:
        yield "Sorry, I couldn't find any relevant information."

async def respond(audio):
    """Handles the full pipeline: transcribe, generate response, and TTS."""
    try:
        # Transcribe audio to text
        user_text = transcribe(audio)

        # Generate response using the language model
        history = []
        response_generator = generate(user_text, history)
        response_text = ""
        for response in response_generator:
            response_text = response

        # Convert the text response to speech
        communicate = edge_tts.Communicate(response_text)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_file:
            tmp_path = tmp_file.name
            await communicate.save(tmp_path)
        return response_text, tmp_path
    except Exception as e:
        return str(e), None

additional_inputs = [
    gr.Slider(
        label="Temperature",
        value=0.9,
        minimum=0.0,
        maximum=1.0,
        step=0.05,
        interactive=True,
        info="Higher values produce more diverse outputs",
    ),
    gr.Slider(
        label="Max new tokens",
        value=512,
        minimum=64,
        maximum=1024,
        step=64,
        interactive=True,
        info="The maximum numbers of new tokens",
    ),
    gr.Slider(
        label="Top-p (nucleus sampling)",
        value=0.90,
        minimum=0.0,
        maximum=1,
        step=0.05,
        interactive=True,
        info="Higher values sample more low-probability tokens",
    ),
    gr.Slider(
        label="Repetition penalty",
        value=1.2,
        minimum=1.0,
        maximum=2.0,
        step=0.05,
        interactive=True,
        info="Penalize repeated tokens",
    )
]

customCSS = """
#component-7 { # this is the default element ID of the chat component
  height: 800px; # adjust the height as needed
  flex-grow: 1;
}
"""

with gr.Blocks(css=customCSS) as demo:
    gr.Markdown("# Speech to text Ai agent. WELCOME ")
    gr.Markdown("Getting real-time updated results for prompts is still proprietary in the face of GPT-4, Co-Pilot etc. This app serves as an open-source alternative for this! this is just a upgrade providing voice-based search comfort for users")

    with gr.Row():
        input_audio = gr.Audio(label="Voice Chat (BETA)", sources="microphone", type="filepath")
        output_text = gr.Textbox(label="Text Response")
        output_audio = gr.Audio(label="Ai agent", type="filepath", interactive=False, autoplay=True, elem_classes="audio")
        gr.Interface(fn=respond, inputs=[input_audio], outputs=[output_text, output_audio], live=True)

    gr.Markdown("## Additional Parameters")
    for slider in additional_inputs:
        slider.render()

demo.queue().launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://84ec35a19982b72354.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
